# Test 13: Prompt Injection with Cosmos DB User Profile

**Goal**: Verify that customer data (name, loyalty tier, location, preferences) is correctly:
1. Loaded from Cosmos DB mock
2. Extracted and flattened in `specialists.py`
3. Passed to Jinja2 templates
4. Rendered in the final system prompt

**Test Scenario**: Sarah Johnson, Gold member from Seattle with athletic style preferences

In [1]:
import sys
import os

# Change to repo root (same pattern as notebook 12)
try:
    os.chdir("../../../")
    target_directory = os.getenv("TARGET_DIRECTORY", os.getcwd())
    if os.path.exists(target_directory):
        os.chdir(target_directory)
        print(f"✅ Changed directory to: {os.getcwd()}")
    else:
        print(f"❌ Directory does not exist: {target_directory}")
except Exception as e:
    print(f"❌ Error changing directory: {e}")

# Add to Python path
backend_path = os.path.join(os.getcwd(), "apps", "rtagent", "backend")
if backend_path not in sys.path:
    sys.path.insert(0, backend_path)

print(f"✅ Python path configured: {backend_path}")

✅ Changed directory to: c:\Users\pablosal\Desktop\art-voice-agent-accelerator
✅ Python path configured: c:\Users\pablosal\Desktop\art-voice-agent-accelerator\apps\rtagent\backend


## Step 1: Connect to Cosmos DB and Load Real User Profile

Query Cosmos DB to get actual user profile structure (not mocked)

In [6]:
# Import Cosmos DB manager - EXACT production pattern
from src.cosmosdb.manager import CosmosDBMongoCoreManager
from utils.ml_logging import get_logger

logger = get_logger("test_prompt_injection")

print("🔗 Initializing Cosmos DB connection (EXACT production pattern)...")
print("   Using: database='retail-db', collection='users'")
try:
    # EXACT production code from main.py lines 428-429
    cosmos_manager = CosmosDBMongoCoreManager(
        database_name="retail-db",
        collection_name="users"
    )
    print(f"✅ Connected to Cosmos DB successfully")
    print(f"   Database: {cosmos_manager.database.name}")
    print(f"   Collection: {cosmos_manager.collection.name}")
except Exception as e:
    print(f"❌ Failed to connect to Cosmos DB: {e}")
    cosmos_manager = None

🔗 Initializing Cosmos DB connection (EXACT production pattern)...
   Using: database='retail-db', collection='users'

   Using: database='retail-db', collection='users'
✅ Connected to Cosmos DB successfully
   Database: retail-db
   Collection: users
✅ Connected to Cosmos DB successfully
   Database: retail-db
   Collection: users


In [7]:
# Query for a user profile - EXACT production pattern from auth.py
test_user_ids = ["sarah_johnson", "michael_chen", "emma_davis"]

cosmos_user = None

if cosmos_manager:
    print("📊 Querying Cosmos DB (EXACT production pattern from auth.py)...")
    for user_id in test_user_ids:
        try:
            # EXACT production code from auth.py line 91
            # Build query exactly like production
            query = {"user_id": user_id}
            
            # Use asyncio.to_thread like production (but synchronously in notebook)
            import asyncio
            try:
                # Try async pattern first
                user_data = await asyncio.to_thread(
                    cosmos_manager.read_document,
                    query
                )
            except RuntimeError:
                # Fallback to sync if no event loop
                user_data = cosmos_manager.read_document(query)
            
            if user_data:
                # Remove MongoDB _id field for cleaner display
                if "_id" in user_data:
                    del user_data["_id"]
                cosmos_user = user_data
                print(f"✅ Found user: {user_id}")
                break
            else:
                print(f"   No user found for: {user_id}")
        except Exception as e:
            print(f"   Query failed for {user_id}: {e}")
            continue
    
    if not cosmos_user:
        print("⚠️  Test users not found, trying to get any user...")
        try:
            # Try to get first document
            user_data = cosmos_manager.read_document({})
            if user_data:
                if "_id" in user_data:
                    del user_data["_id"]
                cosmos_user = user_data
                print(f"✅ Found user: {cosmos_user.get('user_id', 'unknown')}")
        except Exception as e:
            print(f"   Failed: {e}")

# If no real data found, use mock data for testing
if not cosmos_user:
    print("⚠️  Using MOCK data (Cosmos DB unavailable)")
    cosmos_user = {
        "user_id": "sarah_johnson",
        "full_name": "Sarah Johnson",
        "email": "sarah.johnson@example.com",
        "phone": "+1-206-555-0123",
        "dynamics365_data": {
            "customer_id": "CUST-2024-0042",
            "loyalty_tier": "Gold",
            "member_since": "2022-03-15",
            "lifetime_value": 8450.00,
            "total_orders": 47
        },
        "location": {
            "city": "Seattle",
            "state": "WA",
            "country": "USA",
            "timezone": "America/Los_Angeles"
        },
        "preferences": {
            "style": ["athletic", "casual"],
            "brands": ["Nike", "Lululemon", "Adidas"],
            "sizes": {
                "tops": "M",
                "bottoms": "8",
                "shoes": "9"
            },
            "communication": ["email", "sms"]
        },
        "search_history": [
            "running shoes",
            "yoga pants",
            "sports bras",
            "activewear jackets",
            "workout leggings"
        ],
        "conversation_history": []
    }
    print(f"   Mock user: {cosmos_user['user_id']}")
    print(f"   Name: {cosmos_user['full_name']}")
    print(f"   Tier: {cosmos_user['dynamics365_data']['loyalty_tier']}")

📊 Querying Cosmos DB (EXACT production pattern from auth.py)...

✅ Found user: sarah_johnson
✅ Found user: sarah_johnson


## Step 2: Simulate Data Extraction (from specialists.py)

This is the exact logic from `run_shopping_concierge_agent()` that flattens nested data

In [9]:
# Simulate specialists.py data extraction using REAL Cosmos DB data
current_user = cosmos_user  # From cm.get_value_from_corememory("current_user")
user_name = current_user.get("full_name") if isinstance(current_user, dict) else None

# Extract nested data (EXACT logic from specialists.py)
loyalty_data = current_user.get("dynamics365_data", {}) if isinstance(current_user, dict) else {}
location_data = current_user.get("location", {}) if isinstance(current_user, dict) else {}
preferences_data = current_user.get("preferences", {}) if isinstance(current_user, dict) else {}

# Flatten for Jinja2
customer_name = user_name
loyalty_tier = loyalty_data.get("loyalty_tier", "Member")
location = f"{location_data.get('city', 'US')}, {location_data.get('state', '')}".strip(", ") or "US"
style_preferences = ", ".join(preferences_data.get("style", [])) or "Not specified"
recent_searches = ", ".join(current_user.get("search_history", [])[:3]) if isinstance(current_user, dict) else ""

print("✅ Data Extraction (specialists.py logic with REAL data):")
print(f"  customer_name: '{customer_name}'")
print(f"  loyalty_tier: '{loyalty_tier}'")
print(f"  location: '{location}'")
print(f"  style_preferences: '{style_preferences}'")
print(f"  recent_searches: '{recent_searches}'")

# This is what gets passed to respond_kwargs
respond_kwargs = {
    "user_profile": current_user,
    "customer_name": customer_name,
    "loyalty_tier": loyalty_tier,
    "location": location,
    "style_preferences": style_preferences,
    "recent_searches": recent_searches,
}

print("\n📤 respond_kwargs keys:", list(respond_kwargs.keys()))
print(f"   ✅ All 6 required keys present: {len(respond_kwargs) == 6}")

✅ Data Extraction (specialists.py logic with REAL data):
  customer_name: 'Sarah Johnson'
  loyalty_tier: 'Gold'
  location: 'Seattle, WA'
  style_preferences: 'casual, athleisure, minimalist'
  recent_searches: ''
  customer_name: 'Sarah Johnson'
  loyalty_tier: 'Gold'
  location: 'Seattle, WA'
  style_preferences: 'casual, athleisure, minimalist'
  recent_searches: ''

📤 respond_kwargs keys: 

📤 respond_kwargs keys: ['user_profile', 'customer_name', 'loyalty_tier', 'location', 'style_preferences', 'recent_searches']
   ✅ All 6 required keys present: True
['user_profile', 'customer_name', 'loyalty_tier', 'location', 'style_preferences', 'recent_searches']
   ✅ All 6 required keys present: True


In [8]:
# Display full Cosmos DB document structure for debugging
import json

print("📋 FULL COSMOS DB DOCUMENT (Raw JSON):")
print("=" * 80)
print(json.dumps(cosmos_user, indent=2, default=str))
print("=" * 80)

# Verify data type of each field
print("\n🔍 Field Type Verification:")
print(f"  user_id: {type(cosmos_user.get('user_id')).__name__}")
print(f"  full_name: {type(cosmos_user.get('full_name')).__name__}")
print(f"  dynamics365_data: {type(cosmos_user.get('dynamics365_data')).__name__}")
print(f"  location: {type(cosmos_user.get('location')).__name__}")
print(f"  preferences: {type(cosmos_user.get('preferences')).__name__}")
print(f"  search_history: {type(cosmos_user.get('search_history')).__name__}")

📋 FULL COSMOS DB DOCUMENT (Raw JSON):
{
  "user_id": "sarah_johnson",
  "full_name": "Sarah Johnson",
  "age": 28,
  "gender": "female",
  "location": {
    "city": "Seattle",
    "state": "WA",
    "zip": "98101",
    "climate": "temperate_rainy"
  },
  "contact": {
    "email": "sarah.j@email.com",
    "phone": "+12065551234",
    "phone4": "1234"
  },
  "verification": {
    "account_last4": "5678",
    "birthdate_mmdd": "0515"
  },
  "preferences": {
    "style": [
      "casual",
      "athleisure",
      "minimalist"
    ],
    "colors": [
      "navy",
      "grey",
      "black",
      "white"
    ],
    "sizes": {
      "tops": "M",
      "bottoms": "8",
      "shoes": "8"
    },
    "brands": [
      "Nike",
      "Lululemon",
      "Everlane"
    ],
    "fit_preference": "relaxed",
    "price_range": "mid"
  },
  "shopping_patterns": {
    "avg_monthly_spend": 250,
    "purchase_frequency": "bi-weekly",
    "favorite_categories": [
      "Activewear",
      "Casual Tops",
  

## Step 3: Load PromptManager (Production Architecture)

Use the actual PromptManager class that agents use in production

In [11]:
# Import PromptManager - the actual class used by agents in production
# Fix import path - we're in repo root, so we need the full path
import sys
backend_src = os.path.join(os.getcwd(), "apps", "rtagent", "backend", "src")
if backend_src not in sys.path:
    sys.path.insert(0, backend_src)

from agents.artagent.prompt_store.prompt_manager import PromptManager

print("🎨 Initializing PromptManager (production architecture)...")
try:
    # Initialize PromptManager - it will automatically find template directory
    prompt_manager = PromptManager()
    print(f"✅ PromptManager initialized successfully")
    
    # List available templates using Jinja2 env
    templates = prompt_manager.env.list_templates()
    print(f"   Available templates: {len(templates)}")
    for template_name in sorted(templates)[:10]:  # Show first 10
        print(f"     - {template_name}")
except Exception as e:
    print(f"❌ Failed to initialize PromptManager: {e}")
    print(f"   Error type: {type(e).__name__}")
    import traceback
    traceback.print_exc()
    prompt_manager = None

🎨 Initializing PromptManager (production architecture)...
✅ PromptManager initialized successfully
✅ PromptManager initialized successfully
   Available templates: 9
     - fnol_intake_agent.jinja
     - voice_agent_authentication.jinja
     - voice_agent_compliance.jinja
     - voice_agent_fraud_detection.jinja
     - voice_agent_general_info.jinja
   Available templates: 9
     - fnol_intake_agent.jinja
     - voice_agent_authentication.jinja
     - voice_agent_compliance.jinja
     - voice_agent_fraud_detection.jinja
     - voice_agent_general_info.jinja
     - voice_agent_personal_stylist.jinja
     - voice_agent_shopping_concierge.jinja
     - voice_agent_trading.jinja
     - voice_agent_transfer_agency.jinja

     - voice_agent_personal_stylist.jinja
     - voice_agent_shopping_concierge.jinja
     - voice_agent_trading.jinja
     - voice_agent_transfer_agency.jinja


In [12]:
# Render template using PromptManager (exactly like production agents do)
print("🎨 Rendering prompt using PromptManager.get_prompt()...")

if prompt_manager:
    try:
        # This is EXACTLY how agents call it in production (see specialists.py -> base.py)
        template_path = "voice_agent_shopping_concierge.jinja"
        rendered_prompt = prompt_manager.get_prompt(template_path, **respond_kwargs)
        
        print(f"✅ Prompt rendered successfully")
        print(f"   Template: {template_path}")
        print(f"   Prompt length: {len(rendered_prompt)} characters")
        print(f"\n🎨 RENDERED PROMPT (first 1500 chars):\n")
        print("=" * 80)
        print(rendered_prompt[:1500])
        print("=" * 80)
    except Exception as e:
        print(f"❌ Failed to render prompt: {e}")
        print(f"   Error type: {type(e).__name__}")
        import traceback
        traceback.print_exc()
        rendered_prompt = None
else:
    print("❌ PromptManager not available, cannot render template")
    rendered_prompt = None

🎨 Rendering prompt using PromptManager.get_prompt()...

✅ Prompt rendered successfully
   Template: voice_agent_shopping_concierge.jinja
   Prompt length: 9877 characters✅ Prompt rendered successfully
   Template: voice_agent_shopping_concierge.jinja
   Prompt length: 9877 characters

🎨 RENDERED PROMPT (first 1500 chars):



# ROLE
You are ARTAgent's Shopping Concierge—the friendly voice that welcomes customers and helps them discover perfect products.
You're enthusiastic about fashion but never pushy. You listen, understand needs, and connect shoppers with what they're looking for.

# RUNTIME CONTRACT
- One question at a time, short TTS-friendly sentences.
- Always end with punctuation for clear speech synthesis.
- Keep responses under 3 seconds when possible.
- Never mention "tools," "AI," "models," or technical details to customers.
- Stay focused on shopping—products, inventory, pricing, and policies.

The customer has **already been authenticated**. You have access to their profil

## Step 4: Verify Customer Data in Prompt

Check if the customer-specific data appears in the rendered prompt

In [13]:
# Extract the customer data table from rendered prompt
import re

if not rendered_prompt:
    print("❌ No rendered prompt available to verify")
else:
    # Look for the customer data table
    table_pattern = r"\| Customer \| Loyalty Tier \| Location \| Style Preferences \|.*?\|.*?\*\*(.+?)\*\*.*?\*\*(.+?)\*\*.*?\*\*(.+?)\*\*.*?\*\*(.+?)\*\*"
    match = re.search(table_pattern, rendered_prompt, re.DOTALL)
    
    if match:
        print("✅ Customer data table found in prompt:")
        print(f"   Customer: {match.group(1).strip()}")
        print(f"   Loyalty Tier: {match.group(2).strip()}")
        print(f"   Location: {match.group(3).strip()}")
        print(f"   Style Preferences: {match.group(4).strip()}")
        
        # Verify data matches what we extracted
        print(f"\n🔍 Data Match Verification:")
        print(f"   Name matches: {match.group(1).strip() == customer_name}")
        print(f"   Tier matches: {match.group(2).strip() == loyalty_tier}")
        print(f"   Location matches: {match.group(3).strip() == location}")
        print(f"   Style matches: {match.group(4).strip() == style_preferences}")
    else:
        print("❌ Customer data table NOT found in prompt")
        print("\nSearching for individual variables...")
        if customer_name and customer_name in rendered_prompt:
            print(f"  ✅ customer_name found: '{customer_name}'")
        else:
            print(f"  ❌ customer_name NOT found: '{customer_name}'")
        
        if loyalty_tier and loyalty_tier in rendered_prompt:
            print(f"  ✅ loyalty_tier found: '{loyalty_tier}'")
        else:
            print(f"  ❌ loyalty_tier NOT found: '{loyalty_tier}'")
        
        if location and location in rendered_prompt:
            print(f"  ✅ location found: '{location}'")
        else:
            print(f"  ❌ location NOT found: '{location}'")
        
        if style_preferences and style_preferences in rendered_prompt:
            print(f"  ✅ style_preferences found: '{style_preferences}'")
        else:
            print(f"  ❌ style_preferences NOT found: '{style_preferences}'")

✅ Customer data table found in prompt:
   Customer: Sarah Johnson

   Customer: Sarah Johnson
   Loyalty Tier: Gold
   Location: Seattle, WA
   Style Preferences: casual, athleisure, minimalist

🔍 Data Match Verification:   Loyalty Tier: Gold
   Location: Seattle, WA
   Style Preferences: casual, athleisure, minimalist

🔍 Data Match Verification:

   Name matches: True
   Tier matches: True
   Location matches: True
   Style matches: True
   Name matches: True
   Tier matches: True
   Location matches: True
   Style matches: True


## Step 5: Check for VIP Alerts

Gold members should trigger a VIP alert in the prompt

In [ ]:
# Check for VIP alert
if "Gold Member" in rendered_prompt:
    print("✅ Gold Member VIP alert found")
    # Extract the alert text
    alert_pattern = r"(⭐.*?Gold Member.*?)\n"
    alert_match = re.search(alert_pattern, rendered_prompt, re.DOTALL)
    if alert_match:
        print(f"   Alert text: {alert_match.group(1).strip()}")
elif "Platinum" in rendered_prompt:
    print("✅ Platinum VIP alert found")
else:
    print("⚠️  No VIP alert found (member is regular tier)")

## Step 6: Check Recent Searches Context

Verify recent searches appear in the prompt

In [ ]:
# Check for recent searches
if "Recent Interest" in rendered_prompt:
    print("✅ Recent searches section found")
    # Extract recent searches
    searches_pattern = r"Recent Interest.*?:(.+?)\n"
    searches_match = re.search(searches_pattern, rendered_prompt, re.DOTALL)
    if searches_match:
        print(f"   Searches: {searches_match.group(1).strip()}")
else:
    print("⚠️  Recent searches section not found (user may have no search history)")

## Step 7: Test with Different User Profiles

Test edge cases: Platinum member, no style preferences, no recent searches

In [ ]:
# Test Case 1: Platinum member
platinum_user = {
    "full_name": "Michael Chen",
    "dynamics365_data": {"loyalty_tier": "Platinum"},
    "location": {"city": "New York", "state": "NY"},
    "preferences": {"style": ["business_casual", "formal"]},
    "search_history": []
}

# Extract data for Platinum user
loyalty_data_p = platinum_user.get("dynamics365_data", {})
location_data_p = platinum_user.get("location", {})
preferences_data_p = platinum_user.get("preferences", {})

respond_kwargs_p = {
    "user_profile": platinum_user,
    "customer_name": platinum_user["full_name"],
    "loyalty_tier": loyalty_data_p.get("loyalty_tier", "Member"),
    "location": f"{location_data_p.get('city', 'US')}, {location_data_p.get('state', '')}".strip(", "),
    "style_preferences": ", ".join(preferences_data_p.get("style", [])),
    "recent_searches": "",
}

rendered_platinum = template.render(**respond_kwargs_p)

print("💎 PLATINUM MEMBER TEST:")
print(f"   Name: {respond_kwargs_p['customer_name']}")
print(f"   Tier: {respond_kwargs_p['loyalty_tier']}")

if "Platinum member" in rendered_platinum:
    print("   ✅ Platinum VIP alert triggered")
else:
    print("   ❌ Platinum VIP alert NOT found")

if "Recent Interest" in rendered_platinum:
    print("   ⚠️  Recent searches shown (should be empty)")
else:
    print("   ✅ Recent searches section correctly omitted (empty history)")

In [ ]:
# Test Case 2: Member with no preferences
basic_user = {
    "full_name": "Jane Doe",
    "dynamics365_data": {},  # No loyalty tier (defaults to Member)
    "location": {},  # No location (defaults to US)
    "preferences": {},  # No style preferences
    "search_history": []
}

loyalty_data_b = basic_user.get("dynamics365_data", {})
location_data_b = basic_user.get("location", {})
preferences_data_b = basic_user.get("preferences", {})

respond_kwargs_b = {
    "user_profile": basic_user,
    "customer_name": basic_user["full_name"],
    "loyalty_tier": loyalty_data_b.get("loyalty_tier", "Member"),
    "location": f"{location_data_b.get('city', 'US')}, {location_data_b.get('state', '')}".strip(", ") or "US",
    "style_preferences": ", ".join(preferences_data_b.get("style", [])) or "Not specified",
    "recent_searches": "",
}

rendered_basic = template.render(**respond_kwargs_b)

print("👤 BASIC MEMBER TEST:")
print(f"   Name: {respond_kwargs_b['customer_name']}")
print(f"   Tier: {respond_kwargs_b['loyalty_tier']} (default)")
print(f"   Location: {respond_kwargs_b['location']} (default)")
print(f"   Style: {respond_kwargs_b['style_preferences']} (default)")

if "Jane Doe" in rendered_basic:
    print("   ✅ Customer name found")
else:
    print("   ❌ Customer name NOT found")

if "Member" in rendered_basic:
    print("   ✅ Default loyalty tier found")
else:
    print("   ❌ Default loyalty tier NOT found")

## Step 8: Full Prompt Inspection

Display the complete rendered prompt for manual review

In [13]:
if rendered_prompt:
    print("📄 FULL RENDERED PROMPT (Shopping Concierge):")
    print("=" * 80)
    print(rendered_prompt)
    print("=" * 80)
    print(f"\nPrompt length: {len(rendered_prompt)} characters")
    print(f"Prompt lines: {len(rendered_prompt.splitlines())} lines")
    
    # Check for key sections
    print(f"\n📋 Prompt Section Checklist:")
    print(f"   ✅ ROLE section present" if "# ROLE" in rendered_prompt else "   ❌ ROLE section missing")
    print(f"   ✅ RUNTIME CONTRACT present" if "# RUNTIME CONTRACT" in rendered_prompt else "   ❌ RUNTIME CONTRACT missing")
    print(f"   ✅ Customer table present" if "| Customer |" in rendered_prompt else "   ❌ Customer table missing")
    print(f"   ✅ Primary Capabilities present" if "# Primary Capabilities" in rendered_prompt else "   ❌ Primary Capabilities missing")
    print(f"   ✅ Tone & Delivery present" if "# Tone" in rendered_prompt else "   ❌ Tone & Delivery missing")
else:
    print("❌ No rendered prompt available")

📄 FULL RENDERED PROMPT (Shopping Concierge):


# ROLE
You are ARTAgent's Shopping Concierge—the friendly voice that welcomes customers and helps them discover perfect products.
You're enthusiastic about fashion but never pushy. You listen, understand needs, and connect shoppers with what they're looking for.

# RUNTIME CONTRACT
- One question at a time, short TTS-friendly sentences.
- Always end with punctuation for clear speech synthesis.
- Keep responses under 3 seconds when possible.
- Never mention "tools," "AI," "models," or technical details to customers.
- Stay focused on shopping—products, inventory, pricing, and policies.

The customer has **already been authenticated**. You have access to their profile:

| Customer | Loyalty Tier | Location | Style Preferences |
|----------|--------------|----------|-------------------|
| **Sarah Johnson** | **Gold** | **Seattle, WA** | **athletic, casual** |

⛔️ Never ask for their name or account info—already authenticated.


⭐ **Gold Membe

## Summary & Diagnosis

**Expected Results**:
- ✅ Customer name appears in table: "Sarah Johnson"
- ✅ Loyalty tier appears: "Gold"
- ✅ Location appears: "Seattle, WA"
- ✅ Style preferences appear: "athletic, casual"
- ✅ Gold member VIP alert triggered
- ✅ Recent searches shown: "running shoes, yoga pants, sports bras"

**If ANY checks fail**, the issue is in:
1. **Data extraction** (specialists.py) - nested data not flattened correctly
2. **Template syntax** (Jinja2) - variables not referenced correctly
3. **Data passing** (respond_kwargs) - missing keys or incorrect values

**Next Steps**:
- Run this notebook to see actual vs expected behavior
- If prompt looks correct here but agent still doesn't use name, issue is in **model instruction following**, not prompt injection
- May need to add explicit instruction: "Always address customer by name from the table above"